In [3]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import pandas as pd
import seaborn as sns
import numpy as np 
import matplotlib.pyplot as plt
import warnings
import matplotlib
import random
import seaborn as sns
import time
from sklift.metrics import (qini_auc_score)
plt.style.use('fivethirtyeight')
matplotlib.rcParams['figure.figsize'] = [8, 8]
warnings.filterwarnings('ignore')

# logging.basicConfig(level=logging.INFO)
pd.options.display.float_format = '{:.4f}'.format
sns.set(rc={'figure.figsize':(5.7,4.27)})

In [4]:
# 评估函数
def evaluate_bin(t, yf, tau_true, tau_pred):
    pehe = np.sqrt(np.mean(np.square(tau_pred - tau_true)))  # PEHE error

    ate_pred = np.mean(tau_pred)
    atc_pred = np.mean(tau_pred[(1 - t) > 0])
    att_pred = np.mean(tau_pred[t > 0])

    att = np.mean(tau_true[t > 0])
    ate = np.mean(tau_true)

    bias_att = np.abs(att_pred - att)  # the error of att
    bias_ate = np.abs(ate_pred - ate)  # the error of ate

    return {"E_pehe": pehe, "E_att": bias_att, "E_ate": bias_ate}


def eval_mu(y_real, t, yhat_0, yhat_1):
    auc_c = roc_auc_score(y_real, yhat_0, sample_weight = 1-t)
    auc_t = roc_auc_score(y_real, yhat_1, sample_weight = t)
    return {"auc_c":auc_c, "auc_t":auc_t}

def eval_auuc(y_real, pred_tau, t_real):
    return qini_auc_score(y_real, pred_tau, t_real)


# 加载并解析
def load_data(file_path):
    """ Load data set """
    data_in = np.load(file_path)
    data = {'x': data_in['x'], 't': data_in['t'], 'yf': data_in['yf']}
    try:
        data['ycf'] = data_in['ycf']
        data["mu0"] = data_in['mu0']
        data["mu1"] = data_in['mu1']
    except:
        data['ycf'] = None

    try:
        data['e'] = data_in['e']
    except:
        data['e'] = np.ones_like(data_in['yf'])

    try:
        data['tau'] = data_in['tau']
        data['IS_SYNT'] = True
    except:
        data['tau'] = np.array([None])
        data['IS_SYNT'] = False


    data['dim'] = data['x'].shape[1] # 特征维度
    data['n'] = data['x'].shape[0] # 样本数

    return data

In [5]:
name = "lzd"
dd = "real"

train_data_path = "./uplift_data/lzd_real_data_id_v0/real_bin_set_full.5.train.npz".format(str(name+dd))
test_data_path = "./uplift_data/lzd_real_data_id_v0/real_bin_set_full.5.test.npz".format(str(name+dd))

dict_train = load_data(train_data_path)
dict_test = load_data(test_data_path)


print(np.array(dict_train["x"]).shape)
print(np.array(dict_test["x"]).shape)

print(np.array(dict_train["yf"]).shape)
print(np.array(dict_test["yf"]).shape)

(834003, 83, 5)
(181669, 83, 5)
(834003, 5)
(181669, 5)


In [17]:
from causalml.inference.tree import UpliftTreeClassifier, UpliftRandomForestClassifier
from econml.dml import CausalForestDML
    
# save for predictions
train_eval_result={"E_pehe":[], "E_att":[], "E_ate":[], "AUUC":[]}
test_eval_result={"E_pehe":[], "E_att":[], "E_ate":[], "AUUC":[]}
test_random_eval_result={"E_pehe":[], "E_att":[], "E_ate":[], "AUUC":[]}

num_exp = dict_train["x"].shape[2]

include_random=False



# 模拟数据重复生成了多份数据，每一份数据对应一次重复实验
for i_exp in range(num_exp):
    print("running... " + str(i_exp))
    start = time.time()
    # train
    X_train = dict_train["x"][:,:,i_exp]
    yf_train = dict_train["yf"][:,i_exp]
    t_train = dict_train["t"][:,i_exp]

    uplift_model = UpliftRandomForestClassifier(control_name="0",
                                                max_features=10,
                                                min_samples_leaf=100,
                                                random_state=42,
                                                n_estimators=50,
                                                max_depth=5,
                                                n_jobs=8)
    uplift_model.fit(X_train,
                     treatment=t_train.astype(str),
                     y=yf_train)
    # Use defaults
    est = CausalForestDML()
    # Or specify hyperparameters
    # 200, 10, 0.1, 0.5，5
    est = CausalForestDML(n_estimators=200,       
                          max_depth=10,
                          max_features=0.1,
                          max_samples=0.5,
                          subforest_size=10,
                          random_state=42)
    est.fit(yf_train, t_train, X=X_train)

    # predict
    X_test = dict_test["x"][:,:, i_exp]
    yf_test = dict_test["yf"][:, i_exp]
    t_test = dict_test["t"][:, i_exp]

    pred_result = est.effect(X_test)
    _auuc = eval_auuc(yf_test.reshape(-1), pred_result.reshape(-1), t_test.reshape(-1))

    att = np.mean(yf_test.reshape(-1)[t_test.reshape(-1)==1]) - np.mean(yf_test.reshape(-1)[t_test.reshape(-1)==0]) 
    pred_att = np.mean(pred_result.reshape(-1)[t_test.reshape(-1)==1])
    E_att = np.abs(pred_att-att)

    test_eval_result["E_att"].append(E_att)
    test_eval_result["AUUC"].append(_auuc)


print("testing eval results:")
for k in test_eval_result.keys():
    val = np.mean(test_eval_result[k])
    num_exp = len(test_eval_result[k])
    std = np.std(test_eval_result[k])/np.sqrt(num_exp)
    print(k+": %.6f"%val + " +/- %.6f"%std )





running... 0
running... 1
running... 2
running... 3
running... 4
testing eval results:
E_pehe: nan +/- nan
E_att: 0.012280 +/- 0.000250
E_ate: nan +/- nan
AUUC: 0.013207 +/- 0.000848


# BART
https://github.com/JakeColtman/bartpy
pip install git+https://github.com/JakeColtman/bartpy.git --upgrade

In [ ]:
from bartpy.sklearnmodel import SklearnModel

test_eval_result={"E_pehe":[], "E_att":[], "E_ate":[], "AUUC":[]}

for i_exp in range(num_exp):
    print("running... " + str(i_exp))
    start = time.time()
    # train
    X_train = dict_train["x"][:, :, i_exp]
    yf_train = dict_train["yf"][:, i_exp]
    t_train = dict_train["t"][:, i_exp]

    bart_model = SklearnModel(n_trees=20, n_jobs=1)  # Use default parameters
    bart_model.fit(X_train, yf_train)  # Fit the model
 

    # predict
    X_test = dict_test["x"][:, :, i_exp]
    yf_test = dict_test["yf"][:, i_exp]
    t_test = dict_test["t"][:, i_exp]

    pred_result = bart_model.predict(X_test)
   

    _auuc = qini_auc_score(yf_test.reshape(-1), pred_result, t_test.reshape(-1))
    att = np.mean(yf_test.reshape(-1)[t_test.reshape(-1)==1]) - np.mean(yf_test.reshape(-1)[t_test.reshape(-1)==0]) 
    pred_att = np.mean(pred_result[t_test.reshape(-1)==1])
    E_att = np.abs(pred_att-att)
    
    test_eval_result["E_att"].append(E_att)
    test_eval_result["AUUC"].append(_auuc)


print("testing eval results:")
for k in test_eval_result.keys():
    val = np.mean(test_eval_result[k])
    num_exp = len(test_eval_result[k])
    std = np.std(test_eval_result[k])/np.sqrt(num_exp)
    print(k+": %.6f"%val + " +/- %.6f"%std )